In [16]:
import duckdb
import pandas as pd, h3

In [17]:
con = duckdb.connect("ais.duckdb")

In [18]:
# 100 sample only
# df = con.execute("""
#     SELECT *
#     FROM 'output/mmsi_246714000.parquet'
#     WHERE LATITUDE IS NOT NULL
#         AND LONGITUDE IS NOT NULL
#         AND SPEED IS NOT NULL
#         AND HEADING IS NOT NULL
#         AND COURSE IS NOT NULL
#     USING SAMPLE 100;
# """).fetchdf()

df = con.execute("""
    SELECT
        *
    FROM 'output/mmsi_246714000.parquet'
    WHERE LATITUDE IS NOT NULL
        AND LONGITUDE IS NOT NULL
        AND SPEED IS NOT NULL
        AND HEADING IS NOT NULL
        AND COURSE IS NOT NULL;
""").fetchdf()

print(df.head())

             MMSI           TIMESTAMP  LATITUDE   LONGITUDE  SPEED  HEADING  \
0  mmsi-246714000 2023-02-01 00:00:20  1.330283  104.075005   10.6    315.0   
1  mmsi-246714000 2023-02-01 00:00:30  1.330635  104.074662   10.6    315.0   
2  mmsi-246714000 2023-02-01 00:00:41  1.331018  104.074282   10.5    315.0   
3  mmsi-246714000 2023-02-01 00:00:51  1.331365  104.073933   10.5    315.0   
4  mmsi-246714000 2023-02-01 00:00:52  1.331365  104.073933   10.5    315.0   

   COURSE  
0   315.8  
1   315.8  
2   315.9  
3   315.9  
4   315.9  


In [19]:
res = 8

# random top 1 sample
lat, lon = df.iloc[0][['LATITUDE', 'LONGITUDE']]

# basic usage
cell = h3.latlng_to_cell(lat, lon, res)
center = h3.cell_to_latlng(cell)
boundary = h3.cell_to_boundary(cell)
neighbors = h3.grid_disk(cell, 1)
parent = h3.cell_to_parent(cell, 7)
children = h3.cell_to_children(cell, 9)

print("Point:", lat, lon)
print("Cell:", cell)
print("Center:", center)
print("Boundary:", boundary) 
print("Parent:", parent)
print("Children:", children)

Point: 1.3302833333333333 104.075005
Cell: 886526aab7fffff
Center: (1.3327980900871055, 104.07688563161383)
Boundary: ((1.3299130789696743, 104.08103562602518), (1.3349941469297457, 104.08153958011539), (1.337879284335618, 104.0773895356775), (1.335683196409744, 104.07273548047392), (1.3306020006212755, 104.07223164090567), (1.3277170205777185, 104.07638174201013))
Parent: 876526aabffffff
Children: ['896526aab63ffff', '896526aab67ffff', '896526aab6bffff', '896526aab6fffff', '896526aab73ffff', '896526aab77ffff', '896526aab7bffff']


In [20]:
df["H3_R8"] = [h3.latlng_to_cell(lat, lon, 8) for lat, lon in zip(df["LATITUDE"], df["LONGITUDE"])]
df["H3_R9"] = [h3.latlng_to_cell(lat, lon, 9) for lat, lon in zip(df["LATITUDE"], df["LONGITUDE"])]
df["H3_R10"] = [h3.latlng_to_cell(lat, lon, 10) for lat, lon in zip(df["LATITUDE"], df["LONGITUDE"])]
df.head()

,MMSI,TIMESTAMP,LATITUDE,LONGITUDE,SPEED,HEADING,COURSE,H3_R8,H3_R9,H3_R10
0,mmsi-246714000,2023-02-01 00:00:20,1.330283,104.075005,10.6,315.0,315.8,886526aab7fffff,896526aab77ffff,8a6526aab75ffff
1,mmsi-246714000,2023-02-01 00:00:30,1.330635,104.074662,10.6,315.0,315.8,886526aab7fffff,896526aab77ffff,8a6526aab667fff
2,mmsi-246714000,2023-02-01 00:00:41,1.331018,104.074282,10.5,315.0,315.9,886526aab7fffff,896526aab67ffff,8a6526aab667fff
3,mmsi-246714000,2023-02-01 00:00:51,1.331365,104.073933,10.5,315.0,315.9,886526aab7fffff,896526aab67ffff,8a6526aab667fff
4,mmsi-246714000,2023-02-01 00:00:52,1.331365,104.073933,10.5,315.0,315.9,886526aab7fffff,896526aab67ffff,8a6526aab667fff


In [ ]:
# aggregate per hex resolution 8
agg_hex_r8 = (df.groupby("H3_R8")
          .agg(
               point_count=("MMSI","size"),
               # useful when multiple ships in one hex
               # unique_mmsi=("MMSI","nunique"),
               mean_speed=("SPEED","mean"),
               min_speed=("SPEED","min"),
               max_speed=("SPEED","max"),
               std_speed=("SPEED","std"),
               mean_cog=("COURSE","mean"),
               std_cog=("COURSE","std"),
               mean_heading=("HEADING","mean"),
               std_heading=("HEADING","std"),
               ts_min=("TIMESTAMP","min"),
               ts_max=("TIMESTAMP","max"),
          )
         .reset_index())

# aggregate per hex resolution 9
agg_hex_r9 = (df.groupby("H3_R9")
          .agg(
               point_count=("MMSI","size"),
               mean_speed=("SPEED","mean"),
               min_speed=("SPEED","min"),
               max_speed=("SPEED","max"),
               std_speed=("SPEED","std"),
               mean_cog=("COURSE","mean"),
               std_cog=("COURSE","std"),
               mean_heading=("HEADING","mean"),
               std_heading=("HEADING","std"),
               ts_min=("TIMESTAMP","min"),
               ts_max=("TIMESTAMP","max"),
          )
         .reset_index())

# dwell time in minutes
# agg_hex_r9["dwell_minutes"] = (
#     agg_hex_r9["ts_max"] - agg_hex_r9["ts_min"]
# ).dt.total_seconds() / 60

# aggregate per hex resolution 10
agg_hex_r10 = (df.groupby("H3_R10")
     .agg(
          point_count=("MMSI","size"),
          mean_speed=("SPEED","mean"),
          min_speed=("SPEED","min"),
          max_speed=("SPEED","max"),
          std_speed=("SPEED","std"),
          mean_cog=("COURSE","mean"),
          std_cog=("COURSE","std"),
          mean_heading=("HEADING","mean"),
          std_heading=("HEADING","std"),
          ts_min=("TIMESTAMP","min"),
          ts_max=("TIMESTAMP","max"),
     )
     .reset_index())

In [22]:
agg_hex_r8.to_parquet("output/mmsi_246714000_h3_r8.parquet", index=False)
agg_hex_r9.to_parquet("output/mmsi_246714000_h3_r9.parquet", index=False)
agg_hex_r10.to_parquet("output/mmsi_246714000_h3_r10.parquet", index=False)

In [ ]:
# install h3 extension if not already installed
con.execute("INSTALL h3 FROM community;")

In [ ]:
con.execute("LOAD h3;")

In [ ]:
con.execute("ALTER TABLE feb_ais ADD COLUMN h3 VARCHAR;")
con.execute("ALTER TABLE aug_ais ADD COLUMN h3 VARCHAR;")

CatalogException: Catalog Error: Column with name h3 already exists!

In [ ]:
con.execute("""
    UPDATE feb_ais
    SET h3 = h3_latlng_to_cell(LATITUDE, LONGITUDE, 8)
""")

In [ ]:
con.execute("""
    UPDATE aug_ais
    SET h3 = h3_latlng_to_cell(LATITUDE, LONGITUDE, 8)
""")

In [ ]:
con.close()